In [5]:
#download libraries 
import pandas as pd
from datetime import datetime
from datetime import timedelta
import win32com.client as win32
from win32com.client import Dispatch
import os
import openpyxl
from openpyxl import load_workbook
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.utils import get_column_letter
from openpyxl.styles import NamedStyle, Font, Alignment, PatternFill
from openpyxl.styles import Border, Side
from openpyxl.worksheet.table import Table, TableStyleInfo
from openpyxl.cell.cell import Cell
from openpyxl.cell import Cell, MergedCell
from io import BytesIO
import uuid
import schedule
import time

In [6]:
#Define time ranges based on current day
def get_time_range():
    #Current time rounded to the hour
    now = datetime.now().replace(microsecond=0, second=0, minute=0) #Remove microseconds
    weekday = now.weekday() #Monday=0, Sunday=6

    #Debugging: Display current date
    print(f"Current datetime: {now}, Weekday: {weekday}")
    
    #Time ranges
    try:
        #Friday Morning report (07:05 to 13:05)
        if weekday == 4 and (now.hour == 7 and now.minute >= 5 or now.hour < 13): #Friday special setup
            start_time = (now - timedelta(days=1)).replace(hour=19, minute=0, second=0)
            end_time = now.replace(hour=7, minute=0, second=0)
            
        #Friday afternoon report (13:05 to Sunday 07:05)
        elif weekday == 4 and (now.hour == 13 and now.minute >= 5 or now.hour > 13):
            start_time = now.replace(hour=7, minute=0, second=0)
            end_time = now.replace(hour=13, minute=0, second=0)

        #Saturday (No Reports)
        elif weekday == 5: #Saturday (no report day)
            print("No reports scheduled on Saturday.")
            return None, None
            
        #Evening report (19:05 to 07:05 Next Day)
        elif now.hour >= 19 or (now.hour == 19 and now.minute >= 5 or now.hour > 19 or now.hour < 7):
            start_time = now.replace(hour=7, minute=0, second=0)
            end_time = now.replace(hour=19, minute=0, second=0)

        #Morning report (07:05 to 19:05)
        elif now.hour >= 7 or (now.hour == 7 and now.minute >= 5 or now.hour < 19):
            start_time = (now - timedelta(days=1)).replace(hour=19, minute=0,second=0)
            end_time = now.replace(hour=7, minute=0, second=0)

        else:
            print("Current time does not match any operational schedule.")
            return None, None

        #Debugging: Output the assigned time range
        print(f"Start Time: {start_time}, End Time: {end_time}")
        return start_time, end_time
    
    except Exception as e:
        print(f"Error in get_time_range: {e}")
        raise
    
#Set email parameters, send reports
def send_email(output_path, recipient, start_time, end_time, pivot_df):
    try:
        print("Debug: Entering Email Function")

        #Initialize Outlook
        outlook = win32.Dispatch('Outlook.Application') #Open Outlook
        mail = outlook.CreateItem(0) #Create a new email

        #Set up email parameters
        mail.To = "; ".join(recipient)
        mail.Subject = f"Automated 12-Hour Data Report for ({start_time} to {end_time})"

        #Convert pivot table to HTML
        pivot_html = pivot_df.to_html(index=False, border=1)

        #Retrieve Outlook Signature
        signature = "<br><br><br>Data Team Automation</b>"

        #Construct Email body
        email_body = f"""
        <p>Hello,</p>
        <p>Please find the latest automated data report.</p>
        <p>Summary Table:</p>
        {pivot_html}
        <p>Best Regards,</p>
        {signature}
        """

        #Debugging line
        print("Debug: Generated Email Body")
        print(email_body)

        #Assign to HTMLBody
        mail.HTMLBody = email_body

        #Attach the Excel file
        mail.Attachments.Add(output_path)

        #Send email
        mail.Send()
        print("Email sent successfully!")
            
    except Exception as e:
        print(f"Failed to send email: {e}")
    
#Main calculations and formatting
def process_report():
    try:
        start_time, end_time = get_time_range()
        print(f"Debugging Time Range: Start Time: {start_time}, End Time: {end_time}")

        #Load data from Excel source and filtering it
        source_file = r'NETWORK:\...\...\...\Initial_DB.xlsm'
        hidden_sheet = "Closed"
        raw_data = pd.read_excel(source_file, sheet_name=hidden_sheet, engine='openpyxl')

        #Normalize column names in raw_data
        raw_data.columns = raw_data.columns.str.lower().str.replace(' ', '_').str.replace('/', '_').str.replace(':', '')

        #Confirm existing column before filtering
        if 'sfg_top_retest' not in raw_data.columns:
            raise KeyError("'sfg_top_retest' column is missing in raw_data.")

        #Date format
        raw_data['current_time'] = pd.to_datetime(raw_data['current_time'], format='%Y-%m-%d %H:%M')

        #Applying numeric format to 'ticket_lifetime' in raw_data
        raw_data['ticket_lifetime'] = pd.to_timedelta(raw_data['ticket_lifetime'], errors='coerce')

        #Convert 'ticket_lifetim

        #Filtering data
        filtered_df = raw_data[
            (raw_data['current_time'] >= start_time) &
            (raw_data['current_time'] <= end_time)
        ].copy()

        #Expand the pivot table by splitting 'sfg_top_retest' values into separate rows
        filtered_df['sfg_top_retest'] = filtered_df['sfg_top_retest'].str.split(',')
        filtered_df = filtered_df.explode('sfg_top_retest').reset_index(drop=True)

        #Validate Data Type of time values columns
        print(f"ticket_lifetime dtype: {filtered_df['ticket_lifetime'].dtype}")
        print(filtered_df['ticket_lifetime'].head())
        
        #Conversion 'ticket_lifetime' from timedelta to total hours (float)
        filtered_df["ticket_lifetime"] = filtered_df["ticket_lifetime"].dt.total_seconds() / 3600

        #Fill missing values in 'pl' with 'None'
        filtered_df[['pl', 'sfg_top_retest']] = filtered_df[['pl', 'sfg_top_retest']].fillna("None")
        
        #Pivoting into Summary_HCA
        pivot_df = filtered_df.groupby(['pl', 'sfg_top_retest']).agg(
            ticket_qty = ('cause_of_failure', 'count'),
            priority = ('priority', 'first'),
            avrg_time_spend = ('ticket_lifetime', 'mean'),
            total_time_spend = ('ticket_lifetime', 'sum')
        ).reset_index()

        #Reordering pivot_df columns
        column_order = ['pl', 'sfg_top_retest', 'priority', 'ticket_qty', 'avrg_time_spend', 'total_time_spend']
        
        #Chained workflow
        pivot_df = (
            filtered_df.groupby(['pl', 'sfg_top_retest'])
            .agg(
                ticket_qty = ('cause_of_failure', 'count'),
                priority = ('priority', 'first'),
                avrg_time_spend = ('ticket_lifetime', 'mean'),
                total_time_spend = ('ticket_lifetime', 'sum')
            )
            .reset_index()
            [column_order] #Applying column order directly
        )

        #Validation pivot_df
        print(pivot_df.head())

        #Convert float hours to [h]:mm format
        def format_float_to_h_mm(value):
            try:
                hours = int(value)
                minutes = int((value - hours) * 60)
                return f"{hours}:{minutes:02d}"
            except ValueError as e:
                print(f"Error formatting value: {value}, Error: {e}")
                return None
                
        #Apply the formatting functiom to the time columns
        pivot_df['avrg_time_spend'] = pivot_df['avrg_time_spend'].apply(format_float_to_h_mm)
        pivot_df['total_time_spend'] = pivot_df['total_time_spend'].apply(format_float_to_h_mm)
        
        #Debugging final pivot
        print("Formatted time columns:")
        print(pivot_df[['avrg_time_spend', 'total_time_spend']].head())
        
        #Convert 'ticket_lifetime' from float to [h]:mm format
        filtered_df["ticket_lifetime"] = filtered_df["ticket_lifetime"].apply(
            lambda x: f"{int(x):d}:{int((x % 1) * 60):02d}".strip()
        )
        
        #Write filtered data to the "Raw" sheet
        wb = Workbook()
        raw_ws = wb.active
        raw_ws.title = "Raw"

        print(f"Worksheet '{raw_ws.title}' created.")
        
        for r_idx, row in enumerate(dataframe_to_rows(filtered_df, index=False, header=True), start=1):
            for c_idx, value in enumerate(row, start=1):
                raw_ws.cell(row=r_idx, column=c_idx, value=value)

        #Define a [h]:mm number format style if not exists
        if "h_mm" not in wb.named_styles:
            h_mm_style = NamedStyle(name="h_mm", number_format='[h]:mm')
            wb.add_named_style(h_mm_style)

        #Apply the formatting to the 'ticket_lifetime' column in Raw
        for cell in raw_ws["C"][1:]:
            if isinstance(cell.value, str) and ":" in cell.value:
                cell.value = cell.value.replace(" ", "")
                cell.style = "h_mm"

        #Assign the ws after creating Summary_HCA ws
        if "Summary_HCA" not in wb.sheetnames:
            summary_ws = wb.create_sheet("Summary_HCA")
        ws = wb["Summary_HCA"]
        print("Using worksheet:", ws.title)

        #Reordering sheets
        wb._sheets = [wb["Summary_HCA"], wb["Raw"]]

        #Adding table name to the first row
        num_columns = len(pivot_df.columns)
        print(f"Number of columns in pivot table: {num_columns}")

        #Merge cells dynamically based on pivot table width
        summary_ws.merge_cells(start_row=1, start_column=1, end_row=1, end_column=num_columns) #Merge cells
        table_name = f"Summary for 12 hours {start_time.strftime('%Y-%m-%d %H:%M')} - {end_time.strftime('%Y-%m-%d %H:%M')}"
        print(f"Table name: {table_name}")
        
        #Style the table name
        summary_ws.cell(row=1, column=1, value=table_name) #Set the table name in the first merged cell
        title_cell = summary_ws.cell(row=1, column=1)
        title_cell.font = Font(bold=True, size=14)
        title_cell.alignment = Alignment(horizontal="center", vertical="center") #Center alignment
        
        #Function to convert time str to total hours (floats)
        def time_to_float(time_str):
            try:
                if isinstance(time_str, str) and ':' in time_str:
                    hours, minutes = map(int, time_str.split(':'))
                    return hours + minutes / 60
                return float(time_str)
            except Exception as e:
                print(f"Error converting time to float: {time_str}, {e}")
                return 0

        #Applying conversion to time columns
        pivot_df["avrg_time_spend_float"] = pivot_df["avrg_time_spend"].apply(time_to_float)
        pivot_df["total_time_spend_float"] = pivot_df["total_time_spend"].apply(time_to_float)

        #Define the total row
        total_row = {
            'pl': 'Total',
            'sfg_top_retest': '',
            'priority': '',
            'ticket_qty': pivot_df['ticket_qty'].sum(),
            'avrg_time_spend' : pivot_df['avrg_time_spend_float'].mean(),
            'total_time_spend' : pivot_df['total_time_spend_float'].sum(),
        }

        #Back conversion float hours to [h]:mm format
        def float_to_time(hours_float):
            try:
                total_minutes = int(hours_float * 60)
                hours = total_minutes // 60
                minutes = total_minutes % 60
                return f"{hours}:{minutes:02d}"
            except Exception as e:
                print(f"Error converting float to time: {hours_float}, {e}")
                return "0:00"

        #Applying function of back conversion
        total_row["avrg_time_spend"] = float_to_time(total_row["avrg_time_spend"])
        total_row["total_time_spend"] = float_to_time(total_row["total_time_spend"])
                
        #Append the total_row to the rivot_df
        total_row_df = pd. DataFrame([total_row])
        pivot_df = pd.concat([pivot_df, total_row_df], ignore_index=True)

        #Apply bold formatting to the total_row
        total_row_index = len(pivot_df) + 3
        
        for col_idx, value in enumerate(total_row.values(), start=1):
            cell = ws.cell(row=total_row_index, column=col_idx, value=value)
            cell.font = Font(bold=True)

        #Sort the pivot table by 'total_time_spend' in desccending order
        pivot_df = pivot_df.sort_values(by="total_time_spend_float", ascending=False)
        
        #Drop extra float columns
        columns_to_drop = ["avrg_time_spend_float", "total_time_spend_float"]
        pivot_df.drop(columns=[col for col in columns_to_drop if col in pivot_df.columns], inplace=True)

        #Write data rows to the pivot table
        for r_idx, row in enumerate(dataframe_to_rows(pivot_df, index=False, header=False), start=4):
            for c_idx, value in enumerate(row, start=1):
                summary_ws.cell(row=r_idx, column=c_idx, value=value)

        #Write the updated pivot table to 'Summary_HCA' sheet
        headers = list(pivot_df.columns)
        for c_idx, header in enumerate(headers, start=1):
            summary_ws.cell(row=3, column=c_idx, value=header)

        #Adjusting dynamically the cells width
        for col_idx in range(1, summary_ws.max_column + 1):
            col_letter = get_column_letter(col_idx)
            max_length = 0

            header_value = summary_ws.cell(row=3, column=col_idx).value
            if header_value:
                max_length = max(max_length, len(str(header_value)))

            for row in summary_ws.iter_rows(min_col=col_idx, max_col=col_idx, min_row=4, max_row=summary_ws.max_row):
                for cell in row:
                    if cell.value:
                        try:
                            cell_length = len(str(cell.value))
                            max_length = max(max_length, cell_length)
                        except Exception as e:
                            print(f"Error processing cell {cell.coordinate}:{e}")

            #Apply width dynamically based on the max contant length
            summary_ws.column_dimensions[col_letter].width = max_length + 3

        #Center-Align the values
        center_columns = ['priority', 'ticket_qty', 'avrg_time_spend', 'total_time_spend']
        col_indices = {col: idx+1 for idx, col in enumerate(pivot_df.columns) if col in center_columns}

        #Applying center alignment
        for row_idx, row in enumerate(summary_ws.iter_rows(min_row=4, max_row=summary_ws.max_row, min_col=1, max_col=summary_ws.max_column), start=4):
            for col_name, col_idx in col_indices.items():
                cell = summary_ws.cell(row=row_idx, column=col_idx)
                cell.alignment = Alignment(horizontal="center")

        #Applying table style light8
        table_range = f"A3:F{summary_ws.max_row}"
        table = Table(displayName="SummaryTable", ref=table_range)

        #Apply style
        style = TableStyleInfo(
            name = "TableStyleLight8",
            showFirstColumn = False,
            showLastColumn = False,
            showRowStripes = True,
            showColumnStripes = False
        )
        table.tableStyleInfo = style

        #Adding the table to the ws
        summary_ws.add_table(table)

        #Define a thin border style
        thin_border = Border(left=Side(style='thin'),
                            right=Side(style='thin'),
                            top=Side(style='thin'),
                            bottom=Side(style='thin'))
        #Apply borders to all cells in the pivot table (excluding headers)
        for row in summary_ws.iter_rows(min_row=4, max_row=summary_ws.max_row,
                                       min_col=1, max_col=summary_ws.max_column):
            for cell in row:
                cell.border = thin_border
        #Applying the same border to headers
        for cell in summary_ws[3]:
            cell.border = thin_border

        #Save to Excel
        current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M")
        output_path = f"C:\\Users\\...\\Eugen Rovner\\Operational_Table_Report_{current_datetime}.xlsx"
        wb.save(output_path)
        print(f"Report saved to {output_path}")

        #Send Email
        recipient = ["realroer@gmail.com"]
        send_email(output_path, recipient, start_time, end_time, pivot_df)

    except Exception as e:
        print(f"Error in process_report: {e}")

#Schedule tasks
schedule.every().day.at("07:05").do(process_report) #Morning report
schedule.every().day.at("19:05").do(process_report) #Evening report

#Run the scheduler
print("Scheduler started. Waiting for tasks...")
while True:
    schedule.run_pending()
    time.sleep(1)

Scheduler started. Waiting for tasks...


KeyboardInterrupt: 